In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns

In [ ]:
import keras

In [ ]:
from keras.models import Sequential

In [ ]:
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
from keras.optimizers import Adam

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
import tensorflow as tf

In [ ]:
import cv2

In [ ]:
import os

In [ ]:
import sys

In [ ]:
import numpy as np

In [ ]:
#creamos las etiquetas que utilizaremos en la clasificación
labels = ["CLASS_01","CLASS_02","CLASS_03","CLASS_04","CLASS_05","CLASS_06","CLASS_07","CLASS_08"]

In [ ]:
#definimos el tamaño de las imagenes
img_size = 224

In [ ]:
#definimos una función que permita crear nuestro set de datos
def get_data(data_dir):
    data = []
    for label in labels:
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path,img))[...,::-1]
                resized_arr = cv2.resize(img_arr,(img_size, img_size))
                data.append([resized_arr,class_num])
            except Exception as e: print (e)
    return np.array(data,dtype=object)

In [ ]:
train = get_data("./CarneDataset/train")
test = get_data("./CarneDataset/test")

In [ ]:
l=[]
for i in train:
    if(i[1]==0):
        l.append("uno")
    if(i[1]==1):
        l.append("dos")
    if(i[1]==2):
        l.append("tres")
    if(i[1]==3):
        l.append("cuatro")
    if(i[1]==4):
        l.append("cinco")
    if(i[1]==5):
        l.append("seis")
    if(i[1]==6):
        l.append("siete")
    if(i[1]==7):
        l.append("ocho")
    else:
        l.append("otro")
sns.set_style("darkgrid")
sns.countplot(l)

In [ ]:
x_train = []
y_train = []
x_test = []
y_test = []

In [ ]:
for feature, label in train:
    x_train.append(feature)
    y_train.append(label)
for feature, label in test:
    x_test.append(feature)
    y_test.append(label)


In [ ]:
#normalizamos la información
x_train= np.asarray(x_train)/255
x_test= np.asarray(x_test)/255
x_train.reshape(-1, img_size,img_size,1)
y_train=np.asarray(y_train)
x_test.reshape(-1, img_size,img_size,1)
y_test=np.asarray(y_train)



In [ ]:
datagen = ImageDataGenerator(featurewise_center=False,
                            samplewise_center=False,
                            featurewise_std_normalization=False,
                            samplewise_std_normalization=False,
                            zca_whitening=False,
                            rotation_range=30,
                            zoom_range=0.2,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            horizontal_flip=True,
                            vertical_flip=False)

In [ ]:
datagen.fit(x_train)

In [ ]:
#modelo CNN con 3 capas convolucionales y capas de maximix¡zación
model = Sequential()
#primera capa
model.add(Conv2D(32,3,padding="same",activation="relu",input_shape=(224,224,3)))
model.add(MaxPool2D())
#segunda capa
model.add(Conv2D(32,3,padding="same",activation="relu"))
model.add(MaxPool2D())
#tercera capa
model.add(Conv2D(64,3,padding="same",activation="relu"))
model.add(MaxPool2D())
model.add(Dropout(0.4))

#capa de dropout
model.add(Flatten())
model.add(Dense(128,activation="relu"))
model.add(Dense(8,activation="softmax"))

model.summary()




In [ ]:
#optimizamos con Adam y funciòn de pérdida SparseCategoricalCrossentropy
opt=Adam(learning_rate=0.000001)
model.compile(optimizer=opt,loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=["accuracy"])


In [ ]:
history = model.fit(x_train,y_train,epochs=1,validation_data=(x_test,y_test))

In [ ]:
acc=history.history["precision"]
test_acc=history.history["test_precision"]
loss = history.history["perdida"]
test_loss = history.history["test_perdida"]
epochs_range = range(10)

In [ ]:
predicciones=model.predict_classes(x_test)
predicciones=predictions.reshape(1,-1)[0]
print(classification_report(y_test,predicciones, target_names=["CLASS_01","CLASS_02","CLASS_03","CLASS_04","CLASS_05","CLASS_06","CLASS_07","CLASS_08"]))